# **Space X  Falcon 9 First Stage Landing Prediction**

## Introduction
In this Python notebook we will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [2]:
# !pip install sqlalchemy==1.3.9
!pip install ipython-sql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [3]:
%load_ext sql

In [4]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
%sql sqlite:///my_data1.db

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [7]:
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
0,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [8]:
df.columns

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing _Outcome'],
      dtype='object')


##### Display the names of the unique launch sites  in the space mission


In [9]:
%sql select distinct(LAUNCH_SITE) from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [10]:
%sql select * from SPACEXTBL where LAUNCH_SITE like 'CCA%' limit 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [11]:
%sql select sum(PAYLOAD_MASS__KG_) from SPACEXTBL where CUSTOMER = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


##### Display average payload mass carried by booster version F9 v1.1


In [12]:
%sql select avg(PAYLOAD_MASS__KG_) from SPACEXTBL where BOOSTER_VERSION = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2928.4


##### List the date when the first succesful landing outcome in ground pad was acheived.

In [13]:
%%sql
SELECT MIN(Date)
FROM SPACEXTBL
WHERE [Landing _Outcome] = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


MIN(Date)
01-05-2017


##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [14]:
%%sql
SELECT BOOSTER_VERSION
FROM SPACEXTBL
WHERE [LANDING _OUTCOME] = 'Success (drone ship)'
AND 4000< PAYLOAD_MASS__KG_< 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1021.1
F9 FT B1022
F9 FT B1023.1
F9 FT B1026
F9 FT B1029.1
F9 FT B1021.2
F9 FT B1029.2
F9 FT B1036.1
F9 FT B1038.1
F9 B4 B1041.1


##### List the total number of successful and failure mission outcomes


In [15]:
%%sql 
select count(MISSION_OUTCOME) 
from SPACEXTBL where MISSION_OUTCOME = 'Success' or MISSION_OUTCOME = 'Failure (in flight)'

 * sqlite:///my_data1.db
Done.


count(MISSION_OUTCOME)
99


##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [16]:
%%sql 
select BOOSTER_VERSION 
from SPACEXTBL 
where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [17]:
%%sql
select substr(Date, 4, 2) as month, [Landing _Outcome], Booster_Version, Launch_Site
from SPACEXTBL where substr(Date, 7, 4)='2015' and [Landing _Outcome] = 'Failure (drone ship)'

 * sqlite:///my_data1.db
Done.


month,Landing _Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


##### Rank the  count of  successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order.

In [37]:
%%sql 
select * from SPACEXTBL 
where [Landing _Outcome] like 'Success%' and (DATE between '04-06-2010' and '20-03-2017') order by date desc

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
19-02-2017,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
18-10-2020,12:25:57,F9 B5 B1051.6,KSC LC-39A,"Starlink 13 v1.0, Starlink 14 v1.0",15600,LEO,SpaceX,Success,Success
18-08-2020,14:31:00,F9 B5 B1049.6,CCAFS SLC-40,"Starlink 10 v1.0, SkySat-19, -20, -21, SAOCOM 1B",15440,LEO,"SpaceX, Planet Labs, PlanetIQ",Success,Success
18-07-2016,04:45:00,F9 FT B1025.1,CCAFS LC-40,SpaceX CRS-9,2257,LEO (ISS),NASA (CRS),Success,Success (ground pad)
18-04-2018,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362,HEO,NASA (LSP),Success,Success (drone ship)
17-12-2019,00:10:00,F9 B5 B1056.3,CCAFS SLC-40,"JCSat-18 / Kacific 1, Starlink 2 v1.0",6956,GTO,"Sky Perfect JSAT, Kacific 1",Success,Success
16-11-2020,00:27:00,F9 B5B1061.1,KSC LC-39A,"Crew-1, Sentinel-6 Michael Freilich",12500,LEO (ISS),NASA (CCP),Success,Success
15-12-2017,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205,LEO (ISS),NASA (CRS),Success,Success (ground pad)
15-11-2018,20:46:00,F9 B5 B1047.2,KSC LC-39A,Es hail 2,5300,GTO,Es hailSat,Success,Success
14-08-2017,16:31:00,F9 B4 B1039.1,KSC LC-39A,SpaceX CRS-12,3310,LEO (ISS),NASA (CRS),Success,Success (ground pad)
